In [2]:
import numpy as np
import pandas as pd
import cv2
import os
import json
import requests
import wget
import zipfile
import multiprocessing
import datetime
import backtrader as bt
import os, sys, argparse, csv, inspect, time, multiprocessing
import pandas as pd
import backtrader as bt
from tqdm.notebook import tqdm
from pprint import pprint

In [5]:
def FetchAndWriteHistoricalData(symbol, timeframe):
    
    SavePath = "data/Binance/" + str(timeframe) + "MIN/"
    if not os.path.exists(SavePath):
        os.mkdir(SavePath)
        
    HistoricalData = []
    BaseUrl = "http://data.binance.vision/data/spot/monthly/klines/"
    Year = 2021
    Month = 7
    
    while 1:
        try:
            StrMonth = str(StartMonth)
                           
            if len(StrMonth) == 1:
                Month = "0" + Month
                
            url = BaseUrl + symbol + "USDT/" + str(timeframe) + "m/" + symbol + "USDT-" + str(timeframe) + "m-" + str(StartYear) + "-" + Month + ".zip"
                
            #Sample Url : http://data.binance.vision/data/spot/monthly/klines/BNBUSDT/1m/BNBUSDT-1m-2019-01.zip"            
            
            filename = wget.download(url)
            with zipfile.ZipFile(filename, 'r') as zip:
                zip.extractall()     
                
            df = pd.read_csv(filename.split('.')[0] + ".csv",header=None)
            HistoricalData.append(df.values[::-1])                             #storing array in reverse to maintain order of data (as more months' data will be appended to HistoricalData list)
            
            os.remove(filename)
            os.remove(filename.split('.')[0] + ".csv")
        
        except HTTPError:
            break
            
        except Exception as e:
            print(e)
            break
    
    try:
        if not HistoricalData:
            print("No data available for {} on {} hour(s) timeframe".format(symbol,timeframe))
            return
        
        HistoricalData = np.array(HistoricalData)
        df = pd.DataFrame(HistoricalData[::-1])                               #reversing historical data array again so that oldest data is first, latest is last
        df.rename(columns={'1': 'Open',
                           '2': 'High',
                           '3': 'Low',
                           '4': 'Close',
                           '5': 'Volume',
                           '0': 'Date'},inplace=True)
        df['Date'] = pd.to_datetime(df['Date'],unit='s')
        df.drop([6,7,8,9,10,11], axis = 1, inplace=True)
        
        df.to_csv(SavePath + symbol + ".csv", index = False, header = True)
        print("Succesfully Fetched data for {} on {} hour(s) timeframe".format(symbol,timeframe))
        
    except Exception as e:
        print(e)


In [ ]:
timeframes = [1,3,5,15,30]
symbols = ["BTS","BTC","ETH","ADA","ETC","XRP","LTC","XMR","XLM","DOGE",
           "DOT","PPC","DASH","BCH","EOS","LINK","BNB", "REP", "NEO", "TRX"]

SymbolsAndTimeframes = []
for symbol in symbols:
    for timeframe in timeframes:
        SymbolsAndTimeframes.append((symbol,timeframe))

pool = multiprocessing.Pool(12)
pool.starmap(FetchAndWriteHistoricalData, SymbolsAndTimeframes)

In [2]:
HistoricalData = []
filename = wget.download("http://data.binance.vision/data/spot/monthly/klines/BNBUSDT/1m/BNBUSDT-1m-2021-01.zip",out = "temp")
with zipfile.ZipFile(filename, 'r') as zip:
    zip.extractall("temp/")  
df = pd.read_csv(filename.split('.')[0] + ".csv",header=None)
for ticker in df.values[::-1]:
    HistoricalData.append(ticker)
HistoricalData

100% [..........................................................................] 2093612 / 2093612

[array([1.61213754e+12, 4.42614000e+01, 4.44117000e+01, 4.42614000e+01,
        4.43292000e+01, 5.01830900e+03, 1.61213760e+12, 2.22508685e+05,
        7.16000000e+02, 3.38836800e+03, 1.50221072e+05, 0.00000000e+00]),
 array([1.61213748e+12, 4.42416000e+01, 4.42788000e+01, 4.42413000e+01,
        4.42626000e+01, 7.32129000e+02, 1.61213754e+12, 3.24082475e+04,
        1.28000000e+02, 5.88318000e+02, 2.60433877e+04, 0.00000000e+00]),
 array([1.61213742e+12, 4.42269000e+01, 4.42451000e+01, 4.42100000e+01,
        4.42427000e+01, 5.51500000e+02, 1.61213748e+12, 2.43915857e+04,
        1.30000000e+02, 3.39842000e+02, 1.50317416e+04, 0.00000000e+00]),
 array([1.61213736e+12, 4.42592000e+01, 4.42688000e+01, 4.42000000e+01,
        4.42268000e+01, 1.90429800e+03, 1.61213742e+12, 8.42562817e+04,
        1.81000000e+02, 4.51194000e+02, 1.99567274e+04, 0.00000000e+00]),
 array([1.61213730e+12, 4.42398000e+01, 4.42638000e+01, 4.42376000e+01,
        4.42601000e+01, 8.55267000e+02, 1.61213736e+12, 

In [15]:
df = pd.DataFrame(np.array(HistoricalData)[::-1])
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.609459e+12,37.3596,37.3702,37.3381,37.3700,807.624,1.609459e+12,30170.884305,146.0,486.275,18167.580104,0.0
1,1.609459e+12,37.3700,37.4221,37.3487,37.3910,2734.241,1.609459e+12,102217.111475,230.0,1770.005,66157.450698,0.0
2,1.609459e+12,37.3905,37.4020,37.3298,37.3311,778.868,1.609459e+12,29092.024959,141.0,266.496,9953.368200,0.0
3,1.609459e+12,37.3311,37.3367,37.2729,37.2800,890.907,1.609459e+12,33230.401819,156.0,284.896,10625.383029,0.0
4,1.609459e+12,37.2753,37.2997,37.2502,37.2639,483.711,1.609459e+12,18028.653726,126.0,120.471,4491.430352,0.0


In [16]:
df.rename(columns={0: 'Date', 1: 'Open', 2: 'High', 3: 'Low', 4: 'Close', 5: 'Volume'}, inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume,6,7,8,9,10,11
0,1.609459e+12,37.3596,37.3702,37.3381,37.3700,807.624,1.609459e+12,30170.884305,146.0,486.275,18167.580104,0.0
1,1.609459e+12,37.3700,37.4221,37.3487,37.3910,2734.241,1.609459e+12,102217.111475,230.0,1770.005,66157.450698,0.0
2,1.609459e+12,37.3905,37.4020,37.3298,37.3311,778.868,1.609459e+12,29092.024959,141.0,266.496,9953.368200,0.0
3,1.609459e+12,37.3311,37.3367,37.2729,37.2800,890.907,1.609459e+12,33230.401819,156.0,284.896,10625.383029,0.0
4,1.609459e+12,37.2753,37.2997,37.2502,37.2639,483.711,1.609459e+12,18028.653726,126.0,120.471,4491.430352,0.0


In [17]:
df['Date'] = pd.to_datetime(df['Date'],unit='ms')
df.head()

,Date,Open,High,Low,Close,Volume,6,7,8,9,10,11
0,2021-01-01 00:00:00,37.3596,37.3702,37.3381,37.3700,807.624,1.609459e+12,30170.884305,146.0,486.275,18167.580104,0.0
1,2021-01-01 00:01:00,37.3700,37.4221,37.3487,37.3910,2734.241,1.609459e+12,102217.111475,230.0,1770.005,66157.450698,0.0
2,2021-01-01 00:02:00,37.3905,37.4020,37.3298,37.3311,778.868,1.609459e+12,29092.024959,141.0,266.496,9953.368200,0.0
3,2021-01-01 00:03:00,37.3311,37.3367,37.2729,37.2800,890.907,1.609459e+12,33230.401819,156.0,284.896,10625.383029,0.0
4,2021-01-01 00:04:00,37.2753,37.2997,37.2502,37.2639,483.711,1.609459e+12,18028.653726,126.0,120.471,4491.430352,0.0


In [13]:
df["Date"].apply(np.int64)

0        1609459200000
1        1609459260000
2        1609459320000
3        1609459380000
4        1609459440000
             ...      
44635    1612137300000
44636    1612137360000
44637    1612137420000
44638    1612137480000
44639    1612137540000
Name: Date, Length: 44640, dtype: int64

In [28]:
f=open("xyz.txt",'a')
f

<_io.TextIOWrapper name='xyz.txt' mode='a' encoding='cp1252'>

In [29]:
f2=open("xyz.txt",'a')
f2

<_io.TextIOWrapper name='xyz.txt' mode='a' encoding='cp1252'>

In [30]:
f2.write('Hello')

5

In [31]:
f.write('wold')

4

In [33]:
f.close()

In [32]:
f2.close()

In [ ]:
import multiprocessing
def wite(s):
    with open("xyz.txt",'a') as f:
        f.write(s)


if __name__ == "__main__":
    pool = multiprocessing.Pool(12)
    pool.starmap(wite, ["abc","def","ghi","jkl","mno","pqs","tuvw","xyz"])

In [9]:
d = dict([('total', dict([('total', 9), ('open', 0), ('closed', 9)])), ('streak', dict([('won', dict([('current', 2), ('longest', 5)])), ('lost', dict([('current', 0), ('longest', 1)]))])), ('pnl', dict([('gross', dict([('total', 2261.750592042366), ('average', 251.30562133804065)])), ('net', dict([('total', 2261.750592042366), ('average', 251.30562133804065)]))])), ('won', dict([('total', 7), ('pnl', dict([('total', 2667.407145322419), ('average', 381.05816361748845), ('max', 799.9931647699926)]))])), ('lost', dict([('total', 2), ('pnl', dict([('total', -405.6565532800526), ('average', -202.8282766400263), ('max', -247.12530328005266)]))])), ('long', dict([('total', 9), ('pnl', dict([('total', 2261.750592042366), ('average', 251.30562133804065), ('won', dict([('total', 2667.407145322419), ('average', 381.05816361748845), ('max', 799.9931647699926)])), ('lost', dict([('total', -405.6565532800526), ('average', -202.8282766400263), ('max', -247.12530328005266)]))])), ('won', 7), ('lost', 2)])), ('short', dict([('total', 0), ('pnl', dict([('total', 0.0), ('average', 0.0), ('won', dict([('total', 0.0), ('average', 0.0), ('max', 0.0)])), ('lost', dict([('total', 0.0), ('average', 0.0), ('max', 0.0)]))])), ('won', 0), ('lost', 0)])), ('len', dict([('total', 258), ('average', 28.666666666666668), ('max', 95), ('min', 5), ('won', dict([('total', 134), ('average', 19.142857142857142), ('max', 37), ('min', 5)])), ('lost', dict([('total', 124), ('average', 62.0), ('max', 95), ('min', 29)])), ('long', dict([('total', 258), ('average', 28.666666666666668), ('max', 95), ('min', 5), ('won', dict([('total', 134), ('average', 19.142857142857142), ('max', 37), ('min', 5)])), ('lost', dict([('total', 124), ('average', 62.0), ('max', 95), ('min', 29)]))])), ('short', dict([('total', 0), ('average', 0.0), ('max', 0), ('min', 9223372036854775807), ('won', dict([('total', 0), ('average', 0.0), ('max', 0), ('min', 9223372036854775807)])), ('lost', dict([('total', 0), ('average', 0.0), ('max', 0), ('min', 9223372036854775807)]))]))]))])
pprint(d)

{'len': {'average': 28.666666666666668,
         'long': {'average': 28.666666666666668,
                  'lost': {'average': 62.0, 'max': 95, 'min': 29, 'total': 124},
                  'max': 95,
                  'min': 5,
                  'total': 258,
                  'won': {'average': 19.142857142857142,
                          'max': 37,
                          'min': 5,
                          'total': 134}},
         'lost': {'average': 62.0, 'max': 95, 'min': 29, 'total': 124},
         'max': 95,
         'min': 5,
         'short': {'average': 0.0,
                   'lost': {'average': 0.0,
                            'max': 0,
                            'min': 9223372036854775807,
                            'total': 0},
                   'max': 0,
                   'min': 9223372036854775807,
                   'total': 0,
                   'won': {'average': 0.0,
                           'max': 0,
                           'min': 9223372036854775807,
  

In [5]:
num_workers = multiprocessing.cpu_count()
num_workers

12